In [3]:
!pip install guidance

  Obtaining dependency information for guidance from https://files.pythonhosted.org/packages/78/c8/51dcb3767331b3cd4754172e54ff9985d0c87a3da53d4dbf60dfdf8f26a0/guidance-0.0.64-py3-none-any.whl.metadata
  Using cached guidance-0.0.64-py3-none-any.whl.metadata (1.3 kB)
  Using cached diskcache-5.6.1-py3-none-any.whl (45 kB)
  Obtaining dependency information for gptcache from https://files.pythonhosted.org/packages/3d/b2/08e81ec8d1c851a8ccbcec598100920c34f89963c5004a8cb6662a630df0/gptcache-0.1.40-py3-none-any.whl.metadata
  Using cached gptcache-0.1.40-py3-none-any.whl.metadata (23 kB)
  Obtaining dependency information for openai>=0.27.8 from https://files.pythonhosted.org/packages/a0/19/7e39855c63b68ef15b1c2c9e4d941026fb0c94940979e161bb5b470f8455/openai-0.27.9-py3-none-any.whl.metadata
  Using cached openai-0.27.9-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for pyparsing>=3.0.0 from https://files.pythonhosted.org/packages/39/92/8486ede85fcc088f1b3dba4ce92dd29d1

In [40]:
!env | grep OPEN

OPENAI_API_KEY=sk-...


In [52]:
import guidance
import re

# we use GPT-4 here, but you could use gpt-3.5-turbo as well
# guidance.llm = guidance.llms.OpenAI("gpt-4")
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo-0613")

In [53]:
# a custom function we will call in the guidance program
def parse_best(prosandcons, options):
    best = int(re.findall(r'Best=(\d+)', prosandcons)[0])
    return options[best]

# define the guidance program using role tags (like `{{#system}}...{{/system}}`)
create_plan = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{! generate five potential ways to accomplish a goal }}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.
{{~! generate potential options ~}}
Can you please generate one option for how to accomplish this?
Please make the option very short, at most one line.
{{~/user}}

{{#assistant~}}
{{gen 'options' n=5 temperature=1.0 max_tokens=500}}
{{~/assistant}}
{{/block}}

{{! generate pros and cons for each option and select the best option }}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.

Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---{{#each options}}
Option {{@index}}: {{this}}{{/each}}
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.
{{~/user}}

{{#assistant~}}
{{gen 'prosandcons' temperature=0.0 max_tokens=500}}
{{~/assistant}}
{{/block}}

{{! generate a plan to accomplish the chosen option }}
{{#user~}}
I want to {{goal}}.
{{~! Create a plan }}
Here is my plan:
{{parse_best prosandcons options}}
Please elaborate on this plan, and tell me how to best accomplish it.
{{~/user}}

{{#assistant~}}
{{gen 'plan' max_tokens=500}}
{{~/assistant}}''')

# execute the program for a specific goal
out = create_plan(
    goal='read more books',
    parse_best=parse_best # a custom Python function we call in the program
)

Stop program 
 {{#system~}} 
You are a helpful assistant.
 {{~/system}} 


 {{#block hidden=True}} 
 {{#user~}} 
I want to {{goal}} .

Can you please generate one option for how to accomplish this?
Please make the option very short, at most one line.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'options' n=5 temperature=1.0 max_tokens=500}} 
 {{~/assistant}} 
 {{/block}} 


 {{#block hidden=True}} 
 {{#user~}} 
I want to {{goal}} .

Can you please comment on the pros and cons of each of the following options, and then pick the best option?
--- {{#each options}} 
Option {{@index}} : {{this}} {{/each}} 
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'prosandcons' temperature=0.0 max_tokens=500}} 
 {{~/assistant}} 
 {{/block}} 


 {{#user~}} 
I want to {{goal}} .

Here is my plan:
 {{parse_best prosandcons options}} 
Please elaborate on this plan, and tell me how to best accomplish it.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'plan' max_tokens=500}} 
 {{~/assistant}}

In [46]:
# define a guidance program that adapts a proverb
program = guidance("""Tweak this proverb to apply to model instructions instead.

{{proverb}}
- {{book}} {{chapter}}:{{verse}}

UPDATED
Where there is no guidance{{gen 'rewrite' stop="\\n-"}}
- GPT {{#select 'chapter'}}9{{or}}10{{or}}11{{/select}}:{{gen 'verse'}}""")

# execute the program on a specific proverb
executed_program = program(
    proverb="Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.",
    book="Proverbs",
    chapter=11,
    verse=14
)


Stop program Tweak this proverb to apply to model instructions instead.

 {{proverb}} 
- {{book}} {{chapter}} : {{verse}} 

UPDATED
Where there is no guidance {{gen 'rewrite' stop="\n-"}} 
- GPT {{#select 'chapter'}} 9 {{or}} 10 {{or}} 11 {{/select}} : {{gen 'verse'}}